<a href="https://colab.research.google.com/github/malinphy/q_17/blob/main/gpl_nih.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q
!pip install sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.6 MB/s eta 0:00:00


In [2]:
import os 
import json
import pandas as pd 
import numpy as np 
# from data_preprocess import data_loader, column_selection, data_cleaner,delete_multiple_element
import matplotlib.pyplot as plt 

plt.style.use('ggplot')

In [3]:
data_url = 'https://raw.githubusercontent.com/malinphy/datasets/main/patents/nih_df.csv'
df = pd.read_csv(data_url)

In [4]:
df

,title,abstract
0,"Pyrazoline dihydroquinolones, pharmaceutical c...",This disclosure relates to pyrazoline dihydroq...
1,Polyamine compounds targeting the efflux pumps...,Disclosed herein are efflux pump inhibitors an...
2,Processes using nucleoside triphosphates with ...,This invention claims processes that append a ...
3,Mesoporous nitric oxide-releasing silica parti...,"Nitric oxide-releasing materials, methods of m..."
4,Faradic porosity cell,Applicant's faradic porosity cell combines ads...
...,...,...
3238,Engineering t cell receptors,The use of model T cell receptors (TCRs) as sc...
3239,"Superagonists, partial agonists and antagonist...",Novel human interleukin-2 (IL-2) muteins or va...
3240,Mechano-node pore sensing,"Mechano-node-pore sensing (mechano-NPS), is a ..."
3241,Molecular probes for imaging of myelin,A molecular probe for labeling myelin includes...


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'doc2query/msmarco-t5-base-v1'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

In [8]:
index_col = []
questions = []
# for i in range(len(clean_content)):
for i in range(len(df)):
    inputs = tokenizer(df['abstract'][i], return_tensors = 'pt')

    outputs = model.generate(
    input_ids=inputs['input_ids'].cuda(),
    attention_mask=inputs['attention_mask'].cuda(),
    max_length=64,
    do_sample=True,
    top_p=0.95,
    num_return_sequences=5
    )

    index_col.append(np.full(len(outputs),i))
    query_for1 = []
    for i in range(len(outputs)):
        query = tokenizer.decode(outputs[i], skip_special_tokens=True)
        query_for1.append(query)
    questions.append(query_for1)
    # print(f'{i + 1}: {query}')

In [7]:
questions

[['is pyrazoline dihydroquinolone derivable from nmda',
  'what are pyrazoline dihydroquinolone derivatives',
  'what are the pyrazoline dihydroquinolone derivatives',
  'what is pyrazoline dihydroquinolone?',
  'pyrazoline dihydroquinolone uses'],
 ['efflux pump inhibitor',
  'what is efflux pump inhibitor',
  'efflux pump inhibitor definition',
  'what are efflux inhibitors used for',
  'efflux pump inhibitors'],
 ['inventing oligonucleotides with hydroxylamine?',
  'what does onh2 mean',
  'what does onh2 mean in a primer',
  'oligonucleotide primer',
  'what are ohs used for in chemistry'],
 ['what is nitric oxide releasing',
  'uses of nitric oxides',
  'what is nitric oxide release material',
  'what is nitric oxide release materials',
  'what is an nitric oxide release material'],
 ['faradic porosity cell',
  'what is faradic porosity',
  'faradic polyosimetry cell definition',
  'what is faradic porosity cell',
  'what is faradic porosity cell'],
 ['what is the difference betwe